In [1]:
import pandas as  pd 
import numpy as np 


In [2]:
df = pd.read_csv('../datasets/finalTrain.csv')

In [3]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [4]:
df_copy = df.copy()

In [5]:
# Dropping the unique id and type of order columns
df_copy.drop(labels=['ID', 'Delivery_person_ID','Type_of_order'],axis=1 ,inplace=True)
df_copy.columns

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_vehicle', 'multiple_deliveries',
       'Festival', 'City', 'Time_taken (min)'],
      dtype='object')

In [6]:
df_copy.isna().sum()

Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [7]:
## Lets deal with the null values and then proceed 
# Fill missing values in numerical columns with median
numerical_columns = df_copy.select_dtypes(include=np.number).columns
for col in numerical_columns:
    median_value = df_copy[col].median()
    df_copy[col].fillna(median_value, inplace=True)

# Fill missing values in categorical columns with most frequent value
categorical_columns = df_copy.select_dtypes(include='object').columns
for col in categorical_columns:
    most_frequent_value = df_copy[col].mode().iloc[0]
    df_copy[col].fillna(most_frequent_value, inplace=True)

In [8]:
# Convert to datetime format
df_copy['Order_Date'] = pd.to_datetime(df_copy['Order_Date'], format='%d-%m-%Y')

df_copy['year'] = df_copy['Order_Date'].dt.year 
df_copy['month'] = df_copy['Order_Date'].dt.month 
df_copy["day"]=df_copy["Order_Date"].dt.day

In [9]:
# Converting time columns  into hrs and min columns

import datetime
def frac_to_time (column_name:str):
    for i ,value in enumerate(df_copy[column_name]):
        # checking if we are dealing with string and is a fraction time
        if isinstance(value ,str) and '.' in value:
            
            fractional_hours = float(value)
            # Convert fractional hours to seconds
            total_seconds = fractional_hours * 24 * 60 * 60  
            # we are getting extra decimal values so dealing with them
            hours = int(total_seconds // 3600)
            minutes = int((total_seconds % 3600) // 60)
            seconds = int(total_seconds % 60)
            time_value = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
            # Replace the fractional value with the valid time value
            df_copy.at[i, column_name] = time_value



In [10]:
frac_to_time('Time_Orderd') 
frac_to_time('Time_Order_picked')

In [11]:
df_copy['Time_Orderd'] = pd.to_datetime(df_copy['Time_Orderd'], errors='coerce')
df_copy['Time_Order_picked'] = pd.to_datetime(df_copy['Time_Order_picked'], errors='coerce')

C:\Users\kbarv\AppData\Local\Temp\ipykernel_21896\46679988.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_copy['Time_Orderd'] = pd.to_datetime(df_copy['Time_Orderd'], errors='coerce')
C:\Users\kbarv\AppData\Local\Temp\ipykernel_21896\46679988.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_copy['Time_Order_picked'] = pd.to_datetime(df_copy['Time_Order_picked'], errors='coerce')


In [12]:
df_copy.isna().sum()

Delivery_person_Age               0
Delivery_person_Ratings           0
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                     430
Time_Order_picked              1320
Weather_conditions                0
Road_traffic_density              0
Vehicle_condition                 0
Type_of_vehicle                   0
multiple_deliveries               0
Festival                          0
City                              0
Time_taken (min)                  0
year                              0
month                             0
day                               0
dtype: int64

In [13]:
# Filling the null values with most frequent time of order and time of ordered picked
most_frequent_picked_time = df_copy['Time_Order_picked'].mode()[0]
most_frequent_orderd_time = df_copy['Time_Orderd'].mode()[0]

df_copy['Time_Order_picked'] = df_copy['Time_Order_picked'].fillna(most_frequent_picked_time)
df_copy['Time_Orderd'] = df_copy['Time_Orderd'].fillna(most_frequent_orderd_time)
df_copy.isna().sum()

Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Orderd                    0
Time_Order_picked              0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken (min)               0
year                           0
month                          0
day                            0
dtype: int64

In [14]:
# separating hours and minutes into new columns
df_copy['Time_Orderd_hour'] = df_copy['Time_Orderd'].dt.hour.astype(int)
df_copy['Time_Orderd_minute'] = df_copy['Time_Orderd'].dt.minute.astype(int)

df_copy['Time_Order_picked_hour'] = df_copy['Time_Order_picked'].dt.hour.astype(int)
df_copy['Time_Order_picked_minute'] = df_copy['Time_Order_picked'].dt.minute.astype(int)

In [15]:
df_copy.columns

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_vehicle', 'multiple_deliveries',
       'Festival', 'City', 'Time_taken (min)', 'year', 'month', 'day',
       'Time_Orderd_hour', 'Time_Orderd_minute', 'Time_Order_picked_hour',
       'Time_Order_picked_minute'],
      dtype='object')

In [16]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45584 entries, 0 to 45583
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Delivery_person_Age          45584 non-null  float64       
 1   Delivery_person_Ratings      45584 non-null  float64       
 2   Restaurant_latitude          45584 non-null  float64       
 3   Restaurant_longitude         45584 non-null  float64       
 4   Delivery_location_latitude   45584 non-null  float64       
 5   Delivery_location_longitude  45584 non-null  float64       
 6   Order_Date                   45584 non-null  datetime64[ns]
 7   Time_Orderd                  45584 non-null  datetime64[ns]
 8   Time_Order_picked            45584 non-null  datetime64[ns]
 9   Weather_conditions           45584 non-null  object        
 10  Road_traffic_density         45584 non-null  object        
 11  Vehicle_condition            45584 non-nu

In [17]:
df_copy.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,...,Festival,City,Time_taken (min),year,month,day,Time_Orderd_hour,Time_Orderd_minute,Time_Order_picked_hour,Time_Order_picked_minute
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,2022-02-12,2023-08-05 21:55:00,2023-08-05 22:10:00,Fog,...,No,Metropolitian,46,2022,2,12,21,55,22,10
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,2022-02-13,2023-08-05 14:55:00,2023-08-05 15:05:00,Stormy,...,No,Metropolitian,23,2022,2,13,14,55,15,5
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,2022-03-04,2023-08-05 17:30:00,2023-08-05 17:40:00,Sandstorms,...,No,Metropolitian,21,2022,3,4,17,30,17,40
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,2022-02-13,2023-08-05 09:20:00,2023-08-05 09:30:00,Sandstorms,...,No,Metropolitian,20,2022,2,13,9,20,9,30
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,2022-02-14,2023-08-05 19:50:00,2023-08-05 20:05:00,Fog,...,No,Metropolitian,41,2022,2,14,19,50,20,5


In [18]:
# Droping the 'Time_Orderd','Time_Order_picked' and order_date columns
df_copy.drop(["Time_Orderd","Time_Order_picked","Order_Date"],axis=1,inplace=True)


In [19]:
# Separating the features and target columns

X = df_copy.drop(labels=['Time_taken (min)'] ,axis =1)
y = df_copy['Time_taken (min)']

In [20]:
X.columns

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Weather_conditions',
       'Road_traffic_density', 'Vehicle_condition', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'year', 'month', 'day',
       'Time_Orderd_hour', 'Time_Orderd_minute', 'Time_Order_picked_hour',
       'Time_Order_picked_minute'],
      dtype='object')

In [21]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45584 entries, 0 to 45583
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Delivery_person_Age          45584 non-null  float64
 1   Delivery_person_Ratings      45584 non-null  float64
 2   Restaurant_latitude          45584 non-null  float64
 3   Restaurant_longitude         45584 non-null  float64
 4   Delivery_location_latitude   45584 non-null  float64
 5   Delivery_location_longitude  45584 non-null  float64
 6   Weather_conditions           45584 non-null  object 
 7   Road_traffic_density         45584 non-null  object 
 8   Vehicle_condition            45584 non-null  int64  
 9   Type_of_vehicle              45584 non-null  object 
 10  multiple_deliveries          45584 non-null  float64
 11  Festival                     45584 non-null  object 
 12  City                         45584 non-null  object 
 13  year            

In [31]:

#Separating the numerical ,categorical and ordinal
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns
ordinal_col = 'Road_traffic_density'
categorical_cols = categorical_cols.drop(ordinal_col)

# Separate the ordinal column from the rest of the categorical columns
ordinal_cols = [ordinal_col]

In [32]:
df_copy[categorical_cols].columns

Index(['Weather_conditions', 'Type_of_vehicle', 'Festival', 'City'], dtype='object')

In [34]:
print(numerical_cols)
print(categorical_cols)
print(ordinal_cols)

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries', 'year', 'month', 'day', 'Time_Orderd_hour',
       'Time_Orderd_minute', 'Time_Order_picked_hour',
       'Time_Order_picked_minute'],
      dtype='object')
Index(['Weather_conditions', 'Type_of_vehicle', 'Festival', 'City'], dtype='object')
['Road_traffic_density']


In [35]:

from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
from sklearn.preprocessing import LabelEncoder # Label Encoding
from sklearn.preprocessing import OneHotEncoder
## pipelines
from sklearn.pipeline import Pipeline ,make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer

In [36]:
# As Road Traffic is ordinal column we will create custom ranking
traffic_density = ['Low', 'Medium', 'High','Jam']

In [37]:
# Custom ranking for Road_traffic_density column
traffic_density = ['Low', 'Medium', 'High', 'Jam']


# Create the transformers for numerical, ordinal, and nominal data
num_transformer = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_mean=False))

    ]
)

ordinal_transformer = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('ordinalencoder',OrdinalEncoder(categories='auto')),
    ('scaler',StandardScaler())
    ]
)

nominal_transformer = Pipeline(
    steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False))
    ]
   
)

# Create the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transformer', num_transformer, numerical_cols),
        ('ordinal_transformer', ordinal_transformer, ordinal_cols),
        ('nominal_transformer', nominal_transformer, categorical_cols)
    ]
)

In [38]:
## Train test split

from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=30)

In [39]:
type(X_train)

pandas.core.frame.DataFrame

In [40]:

X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())


In [43]:

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [52]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

# Define hyperparameter search space for Decision Tree
dt_param_dist = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define hyperparameter search space for SVM
# svr_param_dist = {
#     'C': uniform(loc=0, scale=10),
#     'kernel': ['linear','rbf']
# }

models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'Elasticnet': ElasticNet(),
    'DecisionTree': DecisionTreeRegressor()
    #'SVR': SVR()
}

model_list = []
r2_list = []

for model_name, model in models.items():
    if model_name in ['DecisionTree', 'SVR']:
        if model_name == 'DecisionTree':
            random_search = RandomizedSearchCV(model, param_distributions=dt_param_dist, n_iter=10)
        elif model_name == 'SVR':
            random_search = RandomizedSearchCV(model, param_distributions=svr_param_dist, n_iter=10)
        
        random_search.fit(X_train, y_train)
        model = random_search.best_estimator_
    else:
        model.fit(X_train, y_train)
        
    y_pred = model.predict(X_test)
    
    mae, rmse, r2_square = evaluate_model(y_test, y_pred)
    
    print(model_name)
    model_list.append(model_name)
    
    print('Model Training Performance')
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("R2 score:", r2_square * 100)
    
    r2_list.append(r2_square)
    
    print('Best Parameters:', random_search.best_params_ if model_name in ['DecisionTree', 'SVM'] else None)
    
    print('=' * 35)
    print('\n')


LinearRegression
Model Training Performance
RMSE: 6.829008400395359
MAE: 5.457626498976309
R2 score: 46.0831249785547
Best Parameters: None


Lasso
Model Training Performance
RMSE: 7.415639556147366
MAE: 5.928449325218744
R2 score: 36.42203195100977
Best Parameters: None


Ridge
Model Training Performance
RMSE: 6.824170658638011
MAE: 5.455235186996807
R2 score: 46.15948848932173
Best Parameters: None


Elasticnet
Model Training Performance
RMSE: 7.290932187294024
MAE: 5.8669146786705335
R2 score: 38.54240844578259
Best Parameters: None


DecisionTree
Model Training Performance
RMSE: 5.087249918932047
MAE: 3.9819348852377536
R2 score: 70.07903707007243
Best Parameters: {'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 10}




## Without Pipline

In [ ]:
# Custom ranking for Road_traffic_density column
traffic_density = ['Low', 'Medium', 'High', 'Jam']

# Numerical preprocessing
num_imputer = SimpleImputer(strategy='median')
num_scaler = StandardScaler()

# Categorical preprocessing
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_label_encoder = LabelEncoder()

# Apply preprocessing to numerical columns
X_train[numerical_cols] = num_imputer.fit_transform(X_train[numerical_cols])
X_train[numerical_cols] = num_scaler.fit_transform(X_train[numerical_cols])

X_test[numerical_cols] = num_imputer.transform(X_test[numerical_cols])
X_test[numerical_cols] = num_scaler.transform(X_test[numerical_cols])

# Handle missing values in categorical columns
X_train[categorical_cols] = cat_imputer.fit_transform(X_train[categorical_cols])
X_test[categorical_cols] = cat_imputer.transform(X_test[categorical_cols])

# Encode Road_traffic_density using custom ordinal encoding
X_train['Road_traffic_density'] = X_train['Road_traffic_density'].fillna('Unknown')
X_train['Road_traffic_density'] = X_train['Road_traffic_density'].map(lambda x: traffic_density.index(x) if x in traffic_density else -1)

X_test['Road_traffic_density'] = X_test['Road_traffic_density'].fillna('Unknown')
X_test['Road_traffic_density'] = X_test['Road_traffic_density'].map(lambda x: traffic_density.index(x) if x in traffic_density else -1)


In [ ]:
X_train['Road_traffic_density'].unique()

array([3, 0, 1, 2], dtype=int64)